In [258]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
import json

In [259]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [260]:
mongo.list_database_names()

['admin',
 'autosaurus',
 'classDB',
 'config',
 'epa',
 'fruits_db',
 'local',
 'met',
 'petsitly_marketing',
 'project3',
 'uk_food']

In [261]:
db = mongo["project3"]

In [262]:
db.list_collection_names()

['emissions']

In [263]:
emissions = db["emissions"]

In [264]:
pprint(emissions.find_one())

{'Cement': '',
 'Coal': '',
 'Country': 'Afghanistan',
 'Flaring': '',
 'Gas': '',
 'ISO 3166-1 alpha-3': 'AFG',
 'Oil': '',
 'Other': '',
 'Per Capita': '',
 'Total': 0,
 'Year': 1750,
 '_id': ObjectId('664ba42777e6e1a7c4a70a4f')}


In [265]:
yearquery = {"Year" : {"$gte" : 1921},
             "Country" : "Global"}
yearquery_df = pd.DataFrame(emissions.find(yearquery))

In [266]:
yearquery_df = yearquery_df[["Country", "Year", "Total"]]
yearquery_df

,Country,Year,Total
0,Global,1921,3089.464239
1,Global,1922,3239.735891
2,Global,1923,3675.398052
3,Global,1924,3686.099236
4,Global,1926,3659.297161
...,...,...,...
96,Global,2017,36096.739276
97,Global,2018,36826.506600
98,Global,2019,37082.558969
99,Global,2020,35264.085734


In [267]:
yearquery_json = yearquery_df.to_json(orient="records")
print(yearquery_json)

[{"Country":"Global","Year":1921,"Total":3089.464239},{"Country":"Global","Year":1922,"Total":3239.735891},{"Country":"Global","Year":1923,"Total":3675.398052},{"Country":"Global","Year":1924,"Total":3686.099236},{"Country":"Global","Year":1926,"Total":3659.297161},{"Country":"Global","Year":1927,"Total":3993.753845},{"Country":"Global","Year":1925,"Total":3725.636685},{"Country":"Global","Year":1928,"Total":3976.070948},{"Country":"Global","Year":1929,"Total":4267.657053},{"Country":"Global","Year":1931,"Total":3516.940495},{"Country":"Global","Year":1930,"Total":3934.730076},{"Country":"Global","Year":1933,"Total":3338.349367},{"Country":"Global","Year":1932,"Total":3170.800548},{"Country":"Global","Year":1934,"Total":3630.765463},{"Country":"Global","Year":1935,"Total":3811.111732},{"Country":"Global","Year":1937,"Total":4469.369267},{"Country":"Global","Year":1936,"Total":4181.088526},{"Country":"Global","Year":1939,"Total":4445.442684},{"Country":"Global","Year":1940,"Total":4854.

In [268]:
parsed = json.loads(yearquery_json)

In [269]:
#https://stackoverflow.com/questions/60029873/pandas-to-json-redundant-backslashes
with open("output.json", "w", encoding="utf8") as output:
    output.write(json.dumps({"data" : parsed}, indent=4))